In [5]:
import requests
import json
import popelines
import os
from datetime import datetime
from pandas.io.json import json_normalize


In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

In [7]:
pope = popelines.popeline(dataset_id='vicky', service_key_file_loc=gbq_key, directory='.', verbose=False)


In [8]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

# Upload Project-Experiment table

In [9]:
# get all projects
response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
j = json.loads(response_proj.text)
project_id = list()
for p in j:
    project_id.append(p['id'])  


In [10]:
requests.get('https://api.optimizely.com/v2/projects', headers=headers)

<Response [200]>

In [11]:
project_list = list()

In [12]:
for project in j:
#     project_list.extend(json_normalize(project).to_dict('records'))
    project_list.append(project)

In [13]:
project_list

[{'account_id': 8725961507,
  'confidence_threshold': 0.9,
  'created': '2018-01-04T20:50:13.101130Z',
  'id': 9965963792,
  'is_classic': False,
  'last_modified': '2018-12-03T18:32:20.382553Z',
  'name': 'Onboarding',
  'platform': 'web',
  'sdks': [],
  'status': 'active',
  'web_snippet': {'code_revision': 191,
   'enable_force_variation': False,
   'exclude_disabled_experiments': False,
   'exclude_names': True,
   'include_jquery': False,
   'ip_anonymization': False,
   'ip_filter': '^(99\\.20\\.23\\.105|76\\.250\\.184\\.81|70\\.166\\.189\\.(3[2-9]|4[0-8]))$',
   'js_file_size': 79948,
   'library': 'none'}},
 {'account_id': 8725961507,
  'confidence_threshold': 0.9,
  'created': '2018-02-22T23:29:15.972680Z',
  'id': 10347872424,
  'is_classic': False,
  'last_modified': '2019-01-18T17:42:11.227784Z',
  'name': 'Corporate Website',
  'platform': 'web',
  'sdks': [],
  'status': 'active',
  'web_snippet': {'code_revision': 1312,
   'enable_force_variation': False,
   'exclude_di

In [ ]:
pope.write_to_json(file_name='projects.json', jayson=project_list, mode='w')
pope.write_to_bq(table_name='projects', file_name='projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


In [ ]:
# get all experiments from one project
p_id = project_id[1]  
params = (
    ('project_id', p_id),
    ('per_page', 100),
) 
response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
j_exp = json.loads(response_exp.text)

In [ ]:
# for exp in j_exp:
#     exp['project_id'] = p_id

In [18]:
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                for j in v:
                    flatten(j, acc, prefix_k)
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [ ]:
# flatten(eg, acc={}, prefix='')

In [ ]:
upload_exp_list = []

In [ ]:
for exp in j_exp:
    flattened_exp = flatten(exp, {}, '')
    upload_exp_list.append(flattened_exp)


In [ ]:
upload_exp_list

In [ ]:
pope.write_to_json(file_name='experiments.json', jayson=upload_exp_list, mode='w')
pope.write_to_bq(table_name='experiments', file_name='experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


In [ ]:
json_normalize(data=j_exp[0],record_path='metrics', meta=['campaign_id', 'description', 'created', 'name'])


# Getting Results (from experiment)

In [14]:
# get all projects
response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
j = json.loads(response_proj.text)
project_id = list()
for p in j:
    project_id.append(p['id'])  

# get all experiments from one project
p_id = project_id[0]  
params = (
    ('project_id', p_id),
    ('per_page', 100),
) 
response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
j_exp = json.loads(response_exp.text)

# get results from one experiment
experiment_id = j_exp[0]['id']
response_res = requests.get(f'https://api.optimizely.com/v2/experiments/{experiment_id}/results', headers=headers)


In [15]:
j_res = json.loads(response_res.text)

In [16]:
j_res.keys()

dict_keys(['confidence_threshold', 'end_time', 'experiment_id', 'metrics', 'reach', 'start_time', 'stats_config'])

In [24]:
j_res['metrics'][0]

{'aggregator': 'unique',
 'event_id': 10757331770,
 'name': 'Create new account button',
 'results': {'10754871910': {'is_baseline': False,
   'level': 'variation',
   'lift': {'confidence_interval': [-0.25586971870854713, 0.17414821658943955],
    'is_significant': False,
    'lift_status': 'worse',
    'significance': 0.0,
    'value': -0.003580937167199094,
    'visitors_remaining': 3239},
   'name': 'Removed phone field',
   'rate': 0.49583333333333335,
   'samples': 240,
   'value': 119.0,
   'variance': 0.2499826388888889,
   'variation_id': '10754871910'},
  '10755800895': {'is_baseline': True,
   'level': 'variation',
   'name': 'Original',
   'rate': 0.49761526232114467,
   'samples': 629,
   'value': 313.0,
   'variance': 0.249994313026203,
   'variation_id': '10755800895'},
  '10766970990': {'is_baseline': False,
   'level': 'variation',
   'lift': {'confidence_interval': [-0.18863950643038613, 0.1409790420200838],
    'is_significant': False,
    'lift_status': 'worse',
   

In [20]:
for m in j_res['metrics']:
    print(m['name'])

Create new account button
Phone
Click Event for .xl .modal-body-inner


In [22]:
flatten(j_res['metrics'][0]['results'], {}, prefix='')

{'10754871910_is_baseline': False,
 '10754871910_level': 'variation',
 '10754871910_lift_is_significant': False,
 '10754871910_lift_lift_status': 'worse',
 '10754871910_lift_significance': 0.0,
 '10754871910_lift_value': -0.003580937167199094,
 '10754871910_lift_visitors_remaining': 3239,
 '10754871910_name': 'Removed phone field',
 '10754871910_rate': 0.49583333333333335,
 '10754871910_samples': 240,
 '10754871910_value': 119.0,
 '10754871910_variance': 0.2499826388888889,
 '10754871910_variation_id': '10754871910',
 '10755800895_is_baseline': True,
 '10755800895_level': 'variation',
 '10755800895_name': 'Original',
 '10755800895_rate': 0.49761526232114467,
 '10755800895_samples': 629,
 '10755800895_value': 313.0,
 '10755800895_variance': 0.249994313026203,
 '10755800895_variation_id': '10755800895',
 '10766970990_is_baseline': False,
 '10766970990_level': 'variation',
 '10766970990_lift_is_significant': False,
 '10766970990_lift_lift_status': 'worse',
 '10766970990_lift_significance'

In [19]:
flatten(j_res['metrics'][0]['results']['10754871910'], {}, prefix='')

{'is_baseline': False,
 'level': 'variation',
 'lift_is_significant': False,
 'lift_lift_status': 'worse',
 'lift_significance': 0.0,
 'lift_value': -0.003580937167199094,
 'lift_visitors_remaining': 3239,
 'name': 'Removed phone field',
 'rate': 0.49583333333333335,
 'samples': 240,
 'value': 119.0,
 'variance': 0.2499826388888889,
 'variation_id': '10754871910'}

In [ ]:
j_res['metrics']

In [ ]:
path = ['metrics']
# 'reach', 'stats_config'

In [ ]:
meta = ['confidence_threshold', 'end_time', 'experiment_id', 'start_time']

In [ ]:
res_df = json_normalize(j_res, record_path=path, meta=meta)

In [ ]:
res_df

# Testing main.py

In [1]:
import requests
import json
import popelines
import os
from datetime import datetime


# a function to detect deeply nested json file, with a mix of dictionaries and lists
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                for j in v:
                    flatten(j, acc, prefix_k)
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [2]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [3]:
# Upload Project-Experiment table
# get all projects
try:
    response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
    j_proj = json.loads(response_proj.text)
    response_proj.raise_for_status()
    print('Successfully read project data.')

except requests.exceptions.HTTPError as err:
    print(err)

Successfully read project data.


In [4]:
# store a list of project metadata
project_list = list()

for project in j_proj:
    p_id = project['id']

    # get all experiments from one project
    params = (
        ('project_id', p_id),
        ('per_page', 100),
    ) 

    try:
        response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
        j_exp = json.loads(response_exp.text)
        response_exp.raise_for_status()
        print('Successfully read experiment data.')
    except requests.exceptions.HTTPError as err:
        print(err)

    # loop for all experiments in this project
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = p_id
        upload_exp_list.append(flatten(exp, {}, ''))

    # upload experiment 
    pope.write_to_json(file_name='experiments.json', jayson=upload_exp_list, mode='w')
    pope.write_to_bq(table_name='experiments', file_name='experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    print(f"Successfully uploaded experiment {exp['id']}")

    project_list.append(project)

INFO:root:Writing experiments to BQ from file experiments.json


Successfully read experiment data.
2019-01-22 12:06:45,839 - root - Writing experiments to BQ from file experiments.json
Successfully uploaded experiment 11992110033


INFO:root:Writing experiments to BQ from file experiments.json


Successfully read experiment data.
2019-01-22 12:09:45,962 - root - Writing experiments to BQ from file experiments.json
Successfully uploaded experiment 13026390548


INFO:root:Writing experiments to BQ from file experiments.json


Successfully read experiment data.
2019-01-22 12:09:54,248 - root - Writing experiments to BQ from file experiments.json
Successfully uploaded experiment 12746930280


INFO:root:Writing experiments to BQ from file experiments.json


Successfully read experiment data.
2019-01-22 12:10:04,376 - root - Writing experiments to BQ from file experiments.json
Successfully uploaded experiment 12900000180


INFO:root:Writing experiments to BQ from file experiments.json


Successfully read experiment data.
2019-01-22 12:10:09,814 - root - Writing experiments to BQ from file experiments.json
Successfully uploaded experiment 11402551541


In [ ]:
project_list